## *change the f_input and f_output run through all blocks to output clean csv

In [18]:
f_input = 'registration_hackthegap_10:4.csv'
f_output = 'Hackthegap_clean.csv'

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import edit_distance

In [20]:
data = pd.DataFrame.from_csv(path= f_input,header=1)
data = data.ix[pd.notnull(data.index),:]
data.index = np.arange(data.shape[0])
data.shape

(108, 26)

In [21]:
majorslist = []
with open('majorslist') as f:
    for line in f.readlines():
        majorslist.append(line.replace('\n','').lower())
        
abbr = {'ce':'computer engineering','cs': 'computer science', \
        'ee': 'electronic engineering','bme':'biomedical engineering','math':'mathematics',\
        'com studies': 'communication studies','business administration':'business'}

In [22]:
# split 

def split(delimiters, string, maxsplit=0):
    regexPattern = '|'.join(map(re.escape, delimiters))
    return re.split(regexPattern, string, maxsplit)

majors = data['What is your major and minor (if applicable)?'].copy()

temp = []
for i in majors:
    phrases = split([';',',','+','and','&','/','.','(',')','-',\
                     'minor','major','Major','Minor',':',' in ','double','Double','with','With'],i)
    phrases = [x.strip().lower() for x in phrases if x is not '' and x is not ' ']
    temp.append(phrases)
majors = temp 

In [23]:
# map
mapping = []
for row in majors:
    temp= []
    for phrase in row:
        
        # deal with abbr.
        if phrase in abbr:
            temp.append(abbr[phrase])
            continue
            
        # compare with each major
        for candidate in majorslist:
            sm = edit_distance.SequenceMatcher(a=phrase, b=candidate)
            if sm.distance()<=2: 
                temp.append(candidate)
                print(phrase,candidate,sm.distance())
                break
    if temp == []: temp.append('undecided')
    mapping.append(temp)

computer engineering computer engineering 0
biomedical engineering biomedical engineering 0
computer science computer science 0
computer engineering computer engineering 0
computer science computer science 0
business business 0
journalism journalism 0
economics economics 0
business business 0
computer engineering computer engineering 0
communication communication 0
journalism journalism 0
german german 0
computer sience computer science 1
international relations international relations  1
economics economics 0
computer science computer science 0
english english 0
computer science computer science 0
biomedical engineering biomedical engineering 0
computer science computer science 0
mathematics mathematics 0
statistics statistics 0
economics economics 0
neuroscience neuroscience 0
public health public health 0
biology biology 0
psychology psychology 0
sociology sociology 0
statistics statistics 0
mathematics mathematics 0
computer science computer science 0
graphic design graphic design 

In [24]:
# intergrate in the dataframe
major1 =[x[0:1] for x in mapping]
major2 =[x[1:2] for x in mapping]
major3 =[x[2:3] for x in mapping]

data = data.assign(major1 =major1,major2=major2,major3=major3)
data = data.drop('What is your major and minor (if applicable)?', axis=1)
data.columns

Index(['What's your first name?',
       'Hey {{answer_57551534}}, nice to meet you.What's your last name?',
       'Which email address shall we use?',
       'What school/college are you enrolled in?',
       'How many hackathons have you participated in before?', 'Python',
       'HTML5/CSS', 'Javascript', 'Java', 'SQL', 'C++', 'Beginner',
       'What is a programming language?',
       'What type of technology are you interested in learning during the hackathon?',
       'Please upload your resume, so we can send them to our sponsors that are looking to hire BU Students',
       'How do you plan to use the technical skills you will learn after hackathon?',
       'Why are you passionate about ending the gender gap in technology?',
       'What is your race?', 'What is your sexual orientation?',
       'Are you a first generation college student?',
       'I hereby allow Hack the Gap to anonymously share my registration information with Hack the Gap and affiliates.',
       'Any ad

In [ ]:
data.to_csv(f_output)